In [1]:
import os
import pickle
import PIL.Image
import numpy as np
import pandas as pd
import csv
import sys
from pathlib import Path
import copy

import matplotlib.pyplot as plt
%matplotlib inline

# path to stylegan encoder on server
sys.path.insert(1, '/home/shavit/APhotoADay/data/stylegan-encoder')

from encoder.generator_model import Generator
import dnnlib
import dnnlib.tflib as tflib
import config

from IPython.display import Markdown, display

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing

from tensorflow import keras

/home/shavit/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shavit/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shavit/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shavit/miniconda3/envs/myenv/lib/python3

In [2]:
URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'

tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)
    
# sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))

generator = Generator(Gs_network, batch_size=1, randomize_noise=False)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
os.getcwd()
# os.chdir("/home/shavit/APhotoADay/True_vs_Learned")

'/home/shavit/APhotoADay/True_vs_Learned'

In [4]:
def generate_image(latent_vector, size):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img.resize((size, size))

In [44]:
data = "kobi"
model = '100_hidden_l1_most_linear.h5'
exper = "100_l1_most_linear_" + data

In [45]:
# Video from given picture, using our data (which specific data exactly?)

os.chdir("/home/shavit/APhotoADay/True_vs_Learned")

# LOAD NL HERE
nl = keras.models.load_model(model)

t_start_photos = []
t_start_ages = []
t_target_ages = []
    
t_first = np.load("regression_data/test/" + data + ".npy").flatten()
# t_first_age = int(input("Enter your age (years): ")) * 365
t_first_age = 20 * 365

for i in range (t_first_age, t_first_age * 2, 10):
    t_start_photos.append(t_first)
    t_start_ages.append(t_first_age)
    t_target_ages.append(i)

t_X = np.zeros(shape = (len(t_start_photos), 512*18+2))
for i in range(len(t_start_photos)):
    t_X[i] = np.concatenate([t_start_ages[i], t_target_ages[i], t_start_photos[i]], axis=None)

results_path = "nl_data/" + exper

t_Y_differences = nl.predict(t_X)

t_Y = []

for i, difference in enumerate(t_Y_differences):
    t_Y.append(difference)

if not os.path.exists(results_path):
    os.mkdir(results_path)

frames = 60

for i, prediction in enumerate(t_Y):
    img = generate_image(prediction, 512)
    img.save(results_path + "/" + str(i) + ".png")
    frames = i

print(frames)

ValueError: Error when checking input: expected dense_32_input to have shape (18434,) but got array with shape (9218,)

In [ ]:
import math
framerate = math.ceil(frames / 6.0)

In [ ]:
# ffmpeg -f image2 -framerate 119 -pattern_type sequence -start_number 0 -r 2 -i %d.png -vcodec libx264 -acodec aac real_broken.mp4
# ffmpeg -i real_broken.mp4 -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p real.mp4
# ffmpeg -itsscale 0.027 -i real.mp4 -c copy real_output.mp4
# os.getcwd()
# First, manually check current path and if it is a father folder, execute chdir 
os.chdir("/home/shavit/APhotoADay/True_vs_Learned/" + results_path)

res1 = os.system("ffmpeg -f image2 -framerate " + str(framerate) + " -pattern_type sequence -start_number 0 -r 2 -i %d.png -vcodec libx264 -acodec aac real_broken.mp4")

if res1 == 0:
    res2 =os.system("ffmpeg -i real_broken.mp4 -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p real.mp4")
    
    if res2 == 0:
        res3 = os.system("ffmpeg -itsscale 0.027 -i real.mp4 -c copy " + exper + ".mp4")
        
        if res3 == 0:
            os.system("rm real_broken.mp4")
            os.system("rm real.mp4")